# Importar bibliotecas

In [11]:
import polars as pl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import plotly.express as px
from IPython.display import Image, display
import os

In [12]:
# Ruta al archivo
ruta_csv = "results.csv"

# Leer el CSV usando Polars
df = pl.read_csv(ruta_csv, encoding='utf8')  # Si hay problemas con acentos, prueba 'latin1' en vez de 'utf8'

#print(df.schema)

# Ver nombres de columnas

# Ver cuántas filas y columnas tiene
#print(f"Número de filas: {df.height}, columnas: {df.width}")
#dict([(i, df.columns[i]) for i in range(len(df.columns))])

idx_questions_per_post_test =  []
idx_questions_per_post_plain_text = []
idx_general_questions = dict([(i, df.columns[i]) for i in range(len(df.columns))][0:11])

for j in range(5):
    idx_questions_per_post_test.append(dict([((i - 2)%3, df.columns[i]) for i in range(len(df.columns))][11+j*3:14+j*3]))
    idx_questions_per_post_plain_text.append(df.columns[-1 - j])


print("idx_questions_per_post_test:", idx_questions_per_post_test)
print("idx_questions_per_post_plain_text:", idx_questions_per_post_plain_text)
print("idx_general_questions:", idx_general_questions)

idx_questions_per_post_test: [{0: '  ¿Qué tan confiable consideras esta noticia?  ', 1: '¿Crees que esta noticia es verdadera o falsa?  ', 2: '¿Piensas que esta noticia fue creada por una persona humana o por una inteligencia artificial?  '}, {0: '  ¿Qué tan confiable consideras esta noticia?  _duplicated_0', 1: '¿Crees que esta noticia es verdadera o falsa?  _duplicated_0', 2: '¿Piensas que esta noticia fue creada por una persona humana o por una inteligencia artificial?  _duplicated_0'}, {0: '  ¿Qué tan confiable consideras esta noticia?  _duplicated_1', 1: '¿Crees que esta noticia es verdadera o falsa?  _duplicated_1', 2: '¿Piensas que esta noticia fue creada por una persona humana o por una inteligencia artificial?  _duplicated_1'}, {0: '  ¿Qué tan confiable consideras esta noticia?  _duplicated_2', 1: '¿Crees que esta noticia es verdadera o falsa?  _duplicated_2', 2: '¿Piensas que esta noticia fue creada por una persona humana o por una inteligencia artificial?  _duplicated_2'}, {

# Análisis de respuesta generales

## 4. ¿Con qué frecuencia estas en entornos digitales?

In [ ]:
# graph index 4 general question with a circular graph
fig = px.pie(df, names=idx_general_questions[4], title=idx_general_questions[4])
fig.update_layout(height=800, width=1200, font=dict(size=20))
display(fig)
fig.write_image("1_4_general_question.png")

## 6. ¿Qué tan común crees que es encontrar noticias falsas en redes sociales?

In [ ]:
# graph index 6 general question with a circular graph
fig = px.pie(df, names=idx_general_questions[6], title=idx_general_questions[6])
fig.update_layout(height=900, width=1200, font=dict(size=20))
display(fig)
fig.write_image("1_6_general_question.png")

## 7. ¿Crees que puedes identificar fácilmente una noticia falsa?  

In [ ]:
# graph index 7 general question with a circular graph (aspect ratio 1:1)
fig = px.pie(df, names=idx_general_questions[7], title=idx_general_questions[7])
scale_factor = 2
fig.update_layout(height=400 * scale_factor, width=550 * scale_factor, font=dict(size=20))
display(fig)
fig.write_image("1_7_general_question.png")

## 9. ¿Sueles verificar la fuente de una noticia antes de compartirla?  

In [ ]:
# graph index 9 general question with a circular graph (aspect ratio 1:1)
fig = px.pie(df, names=idx_general_questions[9], title=idx_general_questions[9])
scale_factor = 2
fig.update_layout(height=400*scale_factor, width=550*scale_factor, font=dict(size=20))
display(fig)
fig.write_image("1_9_general_question.png")

## Prueba: ¿Qué tan confiable consideras esta noticia?  

In [ ]:
# for post section graph the first question of each post

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot with 1 rows and 5 column
fig = make_subplots(rows=1, cols=5, shared_xaxes=True, vertical_spacing=0.05)

for idx, question in enumerate(idx_questions_per_post_test):
    answers = df[idx_questions_per_post_test[idx][0]]
    count_ans = answers.to_list()

    dict_count = dict()
    for i in range(1, 6):
        dict_count[i] = count_ans.count(i)

    dict_count

    # create DataFrame from dict_count
    data = {
        "Respuesta": list(dict_count.keys()),
        "Cantidad": list(dict_count.values())
    }

    df_count = pl.DataFrame(data)
    df_count

    fig.add_trace(
        go.Bar(
            x=df_count['Respuesta'],
            y=df_count['Cantidad'],
            name=f'Noticia {idx + 1}',
            text=df_count['Cantidad'],
            textposition='outside'
        ),
        row=1, col=idx + 1
    )
    
    # calculate mean and median and standard deviation
    mean = np.mean(count_ans)
    median = np.median(count_ans)
    std_dev = np.std(count_ans)
    print(f"Noticia {idx + 1}: {idx_questions_per_post_test[0][0]}")
    print(f"Media: {mean}, Mediana: {median}, Desviación estándar: {std_dev}")
    print("\n")

fig.update_layout(
    title_text=idx_questions_per_post_test[0][0],
    height=800,
    width=1200,
    font=dict(size=20)
)
display(fig)
fig.write_image("1_posts_question.png")

## Prueba: ¿Crees que esta noticia es verdadera o falsa?  

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot with 1 rows and 5 column
fig = make_subplots(rows=1, cols=5, shared_xaxes=True, vertical_spacing=0.02)

dict_correct_answers = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
    4: 0
}

for idx, question in enumerate(idx_questions_per_post_test):
    answers = df[idx_questions_per_post_test[idx][1]]
    count_ans = answers.to_list()

    dict_count = dict()
    for i in ["Verdadera.", "Falsa."]:
        dict_count[i] = count_ans.count(i)
    
    response_correct = {
        "Verdadera.": 0,
        "Falsa.": 1
    }

    # create DataFrame from dict_count
    data = {
        "Respuesta": list([i + ("✅" if dict_correct_answers[idx] != response_correct[i] else "❌") for i in dict_count.keys()]),
        "Cantidad": list(dict_count.values())
    }

    df_count = pl.DataFrame(data)

    # Create a bar trace for the current question
    trace = go.Bar(
        x=df_count['Respuesta'],
        y=df_count['Cantidad'],
        name=f'Noticia {idx + 1}: {idx_questions_per_post_test[idx][0]}',
        text=df_count['Cantidad'],
        textposition='auto'
    )

    # Add the trace to the subplot
    fig.add_trace(trace, row=1, col=idx + 1)

# Update layout
fig.update_layout(
    height=900,  # Adjust the height as needed
    width=1200,  # Adjust the width as needed
    title_text='¿Crees que la noticia es verdadera o falsa?',
    showlegend=False,
    font=dict(size=20)
)
display(fig)

## Prueba: ¿Piensas que esta noticia fue creada por una persona humana o por una inteligencia artificial?  

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=5, shared_xaxes=True, vertical_spacing=0.02)

dict_correct_answers = {
    0: 0,
    1: 1,
    2: 0,
    3: 0,
    4: 1
}

for idx, question in enumerate(idx_questions_per_post_test):
    answers = df[idx_questions_per_post_test[idx][2]]
    count_ans = answers.to_list()

    dict_count = dict()
    for i in ["Humano.", "Inteligencia Artificial."]:
        dict_count[i] = count_ans.count(i)

    response_correct = {
        "Humano.": 0,
        "Inteligencia Artificial.": 1
    }

    abrev_dict = {
        "Humano.": "Hum.",
        "Inteligencia Artificial.": "IA"
    }

    data = {
        "Respuesta": list([abrev_dict[i] + ("✅" if dict_correct_answers[idx] == response_correct[i] else "❌") for i in dict_count.keys()]),
        "Cantidad": list(dict_count.values())
    }

    df_count = pl.DataFrame(data)

    trace = go.Bar(
        x=df_count['Respuesta'],
        y=df_count['Cantidad'],
        name=f'Noticia {idx + 1}: {idx_questions_per_post_test[idx][0]}',
        text=df_count['Cantidad'],
        textposition='auto'
    )
    fig.add_trace(trace, row=1, col=idx + 1)


scale_factor = 3
fig.update_layout(
    height=900,
    width=1200,  
    title_text='¿Quién crees que creó la noticia?',
    font=dict(size=20),
    showlegend=False
)
display(fig)

# Minería de Texto (Topic Modelling)

## Extraer Stop Words

In [13]:
stopwords = []

with open("stop_words_spanish.txt", encoding="utf-8") as sw:
    stopwords = [line.rstrip("\n") for line in sw.readlines()]
print(stopwords)

['algún', 'alguna', 'algunas', 'alguno', 'algunos', 'ambos', 'ampleamos', 'ante', 'antes', 'aquel', 'aquellas', 'aquellos', 'aqui', 'arriba', 'atras', 'bajo', 'bastante', 'bien', 'cada', 'cierta', 'ciertas', 'cierto', 'ciertos', 'como', 'con', 'conseguimos', 'conseguir', 'consigo', 'consigue', 'consiguen', 'consigues', 'cual', 'cuando', 'dentro', 'desde', 'donde', 'dos', 'el', 'ellas', 'ellos', 'empleais', 'emplean', 'emplear', 'empleas', 'empleo', 'en', 'encima', 'entonces', 'entre', 'era', 'eramos', 'eran', 'eras', 'eres', 'es', 'esta', 'estaba', 'estado', 'estais', 'estamos', 'estan', 'estoy', 'fin', 'fue', 'fueron', 'fui', 'fuimos', 'gueno', 'ha', 'hace', 'haceis', 'hacemos', 'hacen', 'hacer', 'haces', 'hago', 'incluso', 'intenta', 'intentais', 'intentamos', 'intentan', 'intentar', 'intentas', 'intento', 'ir', 'la', 'largo', 'las', 'lo', 'los', 'mientras', 'mio', 'modo', 'muchos', 'muy', 'nos', 'nosotros', 'otro', 'para', 'pero', 'podeis', 'podemos', 'poder', 'podria', 'podriais', 

## Topic Modelling

In [17]:
def elect_questions_per_post_topics(df,  idx):
    """
    Dada una pregunta, devuelve las 5 temáticas más relevantes
    """
    result = df[idx_questions_per_post_plain_text[idx]]

    test_docs = [doc if doc is not None else "" for doc in result.to_list()]

    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=stopwords)
    X = vectorizer.fit_transform(test_docs)

    lda = LatentDirichletAllocation(n_components=5, random_state=0)
    lda.fit(X)

    print("Pregunta:", idx_questions_per_post_plain_text[idx])

    def mostrar_temas(model, vectorizer, top_n=5):
        words = vectorizer.get_feature_names_out()
        for idx, topic in enumerate(model.components_):
            print(f"\nTema {idx + 1}:")
            print(", ".join([words[i] for i in topic.argsort()[:-top_n - 1:-1]]))

    mostrar_temas(lda, vectorizer)
    topic_idx = idx
    words = vectorizer.get_feature_names_out()
    topic = lda.components_[topic_idx]
    freqs = {words[i]: topic[i] for i in topic.argsort()[:-30 - 1:-1]}

    total_word_counts = X.toarray().sum(axis=0)
    word_freq_dict = dict(zip(words, total_word_counts))
    for topic_idx, topic_weights in enumerate(lda.components_):
        print(f"\n## Tema {topic_idx + 1}")
        
        top_word_indices = topic_weights.argsort()[:-5 - 1:-1]
        
        topic_data = []
        for word_idx in top_word_indices:
            word = words[word_idx]
            weight = topic_weights[word_idx]
            total_freq = word_freq_dict[word]
            topic_data.append({
                "Palabra": word,
                "Importancia en Tema": f"{weight:.2f}",
                "Frecuencia Total (Corpus)": int(total_freq)
            })
            #display(topic_data[-1])

        topic_df = pl.DataFrame(topic_data)
        display(topic_df)

        os.makedirs("topics", exist_ok=True)
        topic_df.write_csv(f"topics/post_{5-idx}_tema_{topic_idx + 1}.csv")


for idx in range(4, -1, -1):
    elect_questions_per_post_topics(df, idx)

Pregunta: La noticia 1 es Falsa y esta generada por un humano, ¿Qué tan verídica te pareció esta noticia y por que?.

Tema 1:
fuente, falsa, confiable, persona, generar

Tema 2:
falsa, información, presidente, candidata, fuentes

Tema 3:
noticias, falsa, ve, veridica, twitter

Tema 4:
noticia, imágenes, persona, forma, palabras

Tema 5:
verídica, pareció, oficial, fuente, personal

## Tema 1


Palabra,Importancia en Tema,Frecuencia Total (Corpus)
str,str,i64
"""fuente""","""8.15""",10
"""falsa""","""5.91""",10
"""confiable""","""5.16""",6
"""persona""","""3.20""",6
"""generar""","""3.20""",3



## Tema 2


Palabra,Importancia en Tema,Frecuencia Total (Corpus)
str,str,i64
"""falsa""","""2.48""",10
"""información""","""2.20""",3
"""presidente""","""2.20""",2
"""candidata""","""2.20""",2
"""fuentes""","""2.20""",2



## Tema 3


...

Palabra,Importancia en Tema,Frecuencia Total (Corpus)
str,str,i64
"""verídica""","""2.21""",8
"""usuario""","""2.20""",3
"""creí""","""1.20""",2
"""pensaba""","""1.20""",2
"""publicación""","""1.20""",2



## Tema 4


Palabra,Importancia en Tema,Frecuencia Total (Corpus)
str,str,i64
"""noticia""","""3.21""",6
"""opinión""","""3.20""",10
"""información""","""3.20""",3
"""ve""","""3.20""",5
"""verídica""","""3.20""",8



## Tema 5


Palabra,Importancia en Tema,Frecuencia Total (Corpus)
str,str,i64
"""opinión""","""7.20""",10
"""persona""","""6.20""",8
"""falsa""","""5.20""",20
"""odio""","""5.12""",8
"""pareció""","""4.20""",5
